<a href="https://colab.research.google.com/github/EltonWolf/GoogleEarthPythonBootcampAPI-/blob/main/Identifying_Clusters_in_Morocco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Clustering is a technique used to group similar objects or pixels in a spatial dataset based on certain criteria. In Earth Engine, clustering is often used for tasks like image segmentation, where pixels with similar properties are grouped together.

In [1]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=UHzMkoy5Y3edyqSWHrAEWbCVWv3pD2paMSYPprhYkBw&tc=ExdsZ7dnF9wQbJrrfxFkvhZe67RcQfCs3r1xE4F_BsY&cc=HvSD-35M8wsmpxZq5qRhQY1gPneIj3u698ap9AKsgDk

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXlpFzG2JpmVmJZKvqlNh7ajeUzwujm8fuUneojYrdV-ws0nYH-DjPI

Successfully saved authorization token.


In [17]:
countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
Morocco = countries.filter(ee.Filter.eq('country_na', 'Morocco'))

In [18]:
# Load a pre-computed Landsat composite for input
input = ee.Image('LANDSAT/LE7_TOA_1YEAR/2007')

#Define a region in which to generate a sample of the input
region = ee.Geometry.Rectangle(-13.0, 27.5, -3.0, 35.0)

# Make the training dataset
training = input.sample(**{
    'region': region,
    'scale': 30,
    'numPixels': 5000
})

In [19]:
#CLUSTERS
#Initiate the clusterer and train it
clusterer = ee.Clusterer.wekaKMeans(15).train(training)
      #Initializes a K-Means clusterer with 15 clusters.
      #Trains the clusterer using the training dataset.

# Cluster the input using the training clustrer
result = input.cluster(clusterer)
    #Applies the trained clusterer to the input image to perform clustering. Each pixel in the input image is assigned to a cluster.

# Clip the model result
clip = result.clip(Morocco)

In [20]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
my_map = folium.Map(location=[31, -7], zoom_start = 5)

# Add the layer to the map object.
my_map.add_ee_layer(clip.randomVisualizer(), {}, 'clusters')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)